In [0]:
!tar zxvf ./drive/My\ Drive/Colab\ Notebooks/train.tgz
!pip install wandb

In [0]:
!wandb login

# CAPTCHA Recognition - 6 digits

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



# Parsing Data

In [0]:
CLASS_NAMES = np.array([])
for i in range(10):
    CLASS_NAMES =np.append(CLASS_NAMES,chr(ord("0")+i))
for i in range(26):
    CLASS_NAMES = np.append(CLASS_NAMES,chr(ord("A")+i))

In [0]:
#functions
def parse_label(strIN):
    return (strIN==CLASS_NAMES).astype(float)
def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
        #crop to [60,30,3]
    img = tf.io.decode_jpeg(a)
    return img

In [0]:
#training data 01 
train_dir_data01 = os.path.abspath(os.getcwd())+"/train/data01_train/"
data = pd.read_csv("./train/data01_train.csv")
for i in range(len(data)):
    data.iloc[i,1] = list(data.iloc[i,1])
arr = np.zeros([6,50000],str)
for j in range(6):
    for i in range(len(data)):
        arr[j][i] = data.iloc[i,1][j]
data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

for i in range(6):
    data.iloc[:,i+1] = data.iloc[:,i+1].map(parse_label)

TL = tf.convert_to_tensor(list(data.iloc[:,1].values))
TL2 = tf.convert_to_tensor(list(data.iloc[:,2].values))
TL3 = tf.convert_to_tensor(list(data.iloc[:,3].values))
TL4 = tf.convert_to_tensor(list(data.iloc[:,4].values))
TL5 = tf.convert_to_tensor(list(data.iloc[:,5].values))
TL6 = tf.convert_to_tensor(list(data.iloc[:,6].values))

train_dir = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False) 
train_data = train_dir.map(lambda x: readimg_to_tensor(x))
train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
train_data01=tf.data.Dataset.zip((train_data,train_label))

In [0]:
#training data 02
train_dir_data02 = os.path.abspath(os.getcwd())+"/train/data02_train/"
data2 = pd.read_csv("./train/data02_train.csv")
for i in range(len(data2)):
    data2.iloc[i,1] = list(data2.iloc[i,1])
arr = np.zeros([6,50000],str)
for j in range(6):
    for i in range(len(data2)):
        arr[j][i] = data2.iloc[i,1][j]
data2 = data2.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

#to one hot
for i in range(6):
    data2.iloc[:,i+1] = data2.iloc[:,i+1].map(parse_label)

TL = tf.convert_to_tensor(list(data2.iloc[:,1].values))
TL2 = tf.convert_to_tensor(list(data2.iloc[:,2].values))
TL3 = tf.convert_to_tensor(list(data2.iloc[:,3].values))
TL4 = tf.convert_to_tensor(list(data2.iloc[:,4].values))
TL5 = tf.convert_to_tensor(list(data2.iloc[:,5].values))
TL6 = tf.convert_to_tensor(list(data2.iloc[:,6].values))

train_dir2 = tf.data.Dataset.list_files(train_dir_data02+'*.jpg',shuffle=False) 
train_data2 = train_dir2.map(lambda x: readimg_to_tensor(x))
train_label2 = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
train_data02=tf.data.Dataset.zip((train_data2,train_label2))

In [6]:
#number of element
num_elements = tf.data.experimental.cardinality(train_data01).numpy()
num_elements

50000

# Model configeration

In [0]:
import wandb

In [0]:
config_defaults = dict(
    dropout = 0.3,
    conv2D_0_0_size = 32,
    conv2D_0_1_size = 32,
    conv2D_1_0_size = 64,
    conv2D_1_1_size = 64,
    conv2D_2_0_size = 128,
    conv2D_2_1_size = 128,
    conv2D_3_0_size = 256,
    conv2D_3_1_size = 256,
    epochs = 15,
    batch_size = 128
    )

In [8]:
#railway model
print("Creating CNN model...")
inn = Input((60, 200, 3))
out = inn
out = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=256, kernel_size=(3, 3), padding='same',activation='relu')(out)
out = Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Flatten()(out)
sep = Dropout(0.3)(out)
dig1 = Dense(36, name='digit1', activation='softmax')(sep)
dig2 = Dense(36, name='digit2', activation='softmax')(sep)
dig3 = Dense(36, name='digit3', activation='softmax')(sep)
dig4 = Dense(36, name='digit4', activation='softmax')(sep)
dig5 = Dense(36, name='digit5', activation='softmax')(sep)
dig6 = Dense(36, name='digit6', activation='softmax')(sep)
model = tf.keras.models.Model(inputs=inn, outputs=[dig1, dig2,dig3,dig4,dig5,dig6])
model.compile(
    loss=[
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
    ],
    optimizer='adam', 
    metrics=['accuracy'])
model.summary()

Creating CNN model...
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60, 200, 3)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 60, 200, 32)  896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 58, 198, 32)  9248        conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 58, 198, 32)  128         conv2d_1[0][0]                   
________________________________________________________________________

In [0]:
#or import existing one
model = tf.keras.models.load_model("/content/data01_20200521.h5")

# Fit model

In [11]:
wandb.init(project="my-project",config=config_defaults)
config = wandb.config

In [16]:
validation_split

(0.1,)

In [0]:
epochs=5
batch_size=128
validation_split=0.1
split = int(num_elements*validation_split)
train_data_gen=iter(train_data01.shuffle(10000).batch(50000))

img , lb = next(train_data_gen)
img_train = img[:-split]
lb_train = (lb[0][:-split],lb[1][:-split],lb[2][:-split],lb[3][:-split],lb[4][:-split],lb[5][:-split])
img_test = img[-split:]
lb_test = (lb[0][-split:],lb[1][-split:],lb[2][-split:],lb[3][-split:],lb[4][-split:],lb[5][-split:])


In [19]:
history = model.fit(
       img_train,
       lb_train,
       validation_data=(img_test, lb_test),
       batch_size=batch_size,
       epochs=config.epochs,
       verbose=1,
       callbacks=[
          wandb.keras.WandbCallback(
              data_type="image",
              labels=CLASS_NAMES)]
       )

Epoch 1/15
352/352 [==============================] - ETA: 0s - loss: 21.1692 - digit1_loss: 3.5258 - digit2_loss: 3.5148 - digit3_loss: 3.5254 - digit4_loss: 3.5280 - digit5_loss: 3.5338 - digit6_loss: 3.5414 - digit1_accuracy: 0.0492 - digit2_accuracy: 0.0500 - digit3_accuracy: 0.0474 - digit4_accuracy: 0.0489 - digit5_accuracy: 0.0485 - digit6_accuracy: 0.0461

wandb: WARNING No validation_data set, pass a generator to the callback.


352/352 [==============================] - 32s 91ms/step - loss: 21.1692 - digit1_loss: 3.5258 - digit2_loss: 3.5148 - digit3_loss: 3.5254 - digit4_loss: 3.5280 - digit5_loss: 3.5338 - digit6_loss: 3.5414 - digit1_accuracy: 0.0492 - digit2_accuracy: 0.0500 - digit3_accuracy: 0.0474 - digit4_accuracy: 0.0489 - digit5_accuracy: 0.0485 - digit6_accuracy: 0.0461 - val_loss: 27.1622 - val_digit1_loss: 4.4465 - val_digit2_loss: 4.7332 - val_digit3_loss: 4.4924 - val_digit4_loss: 4.5667 - val_digit5_loss: 4.5415 - val_digit6_loss: 4.3819 - val_digit1_accuracy: 0.0332 - val_digit2_accuracy: 0.0280 - val_digit3_accuracy: 0.0234 - val_digit4_accuracy: 0.0280 - val_digit5_accuracy: 0.0334 - val_digit6_accuracy: 0.0348
Epoch 2/15
352/352 [==============================] - ETA: 0s - loss: 12.4194 - digit1_loss: 2.0411 - digit2_loss: 1.9815 - digit3_loss: 2.0148 - digit4_loss: 2.0752 - digit5_loss: 2.1495 - digit6_loss: 2.1571 - digit1_accuracy: 0.3659 - digit2_accuracy: 0.3792 - digit3_accuracy: 0.

wandb: WARNING No validation_data set, pass a generator to the callback.


352/352 [==============================] - 31s 87ms/step - loss: 12.4194 - digit1_loss: 2.0411 - digit2_loss: 1.9815 - digit3_loss: 2.0148 - digit4_loss: 2.0752 - digit5_loss: 2.1495 - digit6_loss: 2.1571 - digit1_accuracy: 0.3659 - digit2_accuracy: 0.3792 - digit3_accuracy: 0.3711 - digit4_accuracy: 0.3562 - digit5_accuracy: 0.3404 - digit6_accuracy: 0.3350 - val_loss: 4.6725 - val_digit1_loss: 0.7836 - val_digit2_loss: 0.7312 - val_digit3_loss: 0.7057 - val_digit4_loss: 0.7707 - val_digit5_loss: 0.8029 - val_digit6_loss: 0.8785 - val_digit1_accuracy: 0.7608 - val_digit2_accuracy: 0.7746 - val_digit3_accuracy: 0.7832 - val_digit4_accuracy: 0.7698 - val_digit5_accuracy: 0.7614 - val_digit6_accuracy: 0.7364
Epoch 3/15
352/352 [==============================] - ETA: 0s - loss: 2.7219 - digit1_loss: 0.4641 - digit2_loss: 0.4299 - digit3_loss: 0.4026 - digit4_loss: 0.4636 - digit5_loss: 0.4891 - digit6_loss: 0.4725 - digit1_accuracy: 0.8540 - digit2_accuracy: 0.8663 - digit3_accuracy: 0.87

...

352/352 [==============================] - 31s 87ms/step - loss: 0.2554 - digit1_loss: 0.0446 - digit2_loss: 0.0395 - digit3_loss: 0.0369 - digit4_loss: 0.0408 - digit5_loss: 0.0466 - digit6_loss: 0.0471 - digit1_accuracy: 0.9862 - digit2_accuracy: 0.9876 - digit3_accuracy: 0.9883 - digit4_accuracy: 0.9880 - digit5_accuracy: 0.9859 - digit6_accuracy: 0.9851 - val_loss: 0.3406 - val_digit1_loss: 0.0554 - val_digit2_loss: 0.0515 - val_digit3_loss: 0.0453 - val_digit4_loss: 0.0566 - val_digit5_loss: 0.0676 - val_digit6_loss: 0.0643 - val_digit1_accuracy: 0.9844 - val_digit2_accuracy: 0.9846 - val_digit3_accuracy: 0.9866 - val_digit4_accuracy: 0.9842 - val_digit5_accuracy: 0.9816 - val_digit6_accuracy: 0.9836
Epoch 14/15
352/352 [==============================] - ETA: 0s - loss: 0.2472 - digit1_loss: 0.0408 - digit2_loss: 0.0380 - digit3_loss: 0.0365 - digit4_loss: 0.0377 - digit5_loss: 0.0502 - digit6_loss: 0.0439 - digit1_accuracy: 0.9877 - digit2_accuracy: 0.9883 - digit3_accuracy: 0.98

wandb: WARNING No validation_data set, pass a generator to the callback.


352/352 [==============================] - 30s 87ms/step - loss: 0.2472 - digit1_loss: 0.0408 - digit2_loss: 0.0380 - digit3_loss: 0.0365 - digit4_loss: 0.0377 - digit5_loss: 0.0502 - digit6_loss: 0.0439 - digit1_accuracy: 0.9877 - digit2_accuracy: 0.9883 - digit3_accuracy: 0.9890 - digit4_accuracy: 0.9881 - digit5_accuracy: 0.9843 - digit6_accuracy: 0.9868 - val_loss: 0.2816 - val_digit1_loss: 0.0395 - val_digit2_loss: 0.0402 - val_digit3_loss: 0.0342 - val_digit4_loss: 0.0513 - val_digit5_loss: 0.0570 - val_digit6_loss: 0.0594 - val_digit1_accuracy: 0.9890 - val_digit2_accuracy: 0.9888 - val_digit3_accuracy: 0.9884 - val_digit4_accuracy: 0.9870 - val_digit5_accuracy: 0.9850 - val_digit6_accuracy: 0.9846
Epoch 15/15
352/352 [==============================] - ETA: 0s - loss: 0.2281 - digit1_loss: 0.0386 - digit2_loss: 0.0330 - digit3_loss: 0.0313 - digit4_loss: 0.0387 - digit5_loss: 0.0442 - digit6_loss: 0.0424 - digit1_accuracy: 0.9887 - digit2_accuracy: 0.9901 - digit3_accuracy: 0.99

wandb: WARNING No validation_data set, pass a generator to the callback.


352/352 [==============================] - 30s 86ms/step - loss: 0.2281 - digit1_loss: 0.0386 - digit2_loss: 0.0330 - digit3_loss: 0.0313 - digit4_loss: 0.0387 - digit5_loss: 0.0442 - digit6_loss: 0.0424 - digit1_accuracy: 0.9887 - digit2_accuracy: 0.9901 - digit3_accuracy: 0.9900 - digit4_accuracy: 0.9883 - digit5_accuracy: 0.9860 - digit6_accuracy: 0.9872 - val_loss: 0.5861 - val_digit1_loss: 0.0986 - val_digit2_loss: 0.0816 - val_digit3_loss: 0.0791 - val_digit4_loss: 0.1036 - val_digit5_loss: 0.1084 - val_digit6_loss: 0.1148 - val_digit1_accuracy: 0.9750 - val_digit2_accuracy: 0.9780 - val_digit3_accuracy: 0.9770 - val_digit4_accuracy: 0.9726 - val_digit5_accuracy: 0.9698 - val_digit6_accuracy: 0.9718


In [0]:
model.save('data02_20200522_105epochs.h5')

# Testing
